In [1]:
import json
import requests
from src import utils
import time
import os
from pprint import pprint
import random
from tqdm import tqdm

In [2]:
api_path = '/userhomes/philhoon/kt-ai-challenge/api-peft3.json'

In [3]:
api = utils.read_json_file(api_path)

In [4]:
# output_path = '/userhomes/philhoon/kt-ai-challenge/result'

In [5]:
# os.makedirs(output_path, exist_ok=True)
# output_file_name = 'result-' + input_file.split('/')[-1]
# output_file = output_path + '/' + output_file_name
# print(f'output_file : {output_file}')

In [6]:
# peft tunning
# peft1 = '/userhomes/philhoon/kt-ai-challenge/result/kor-gsm8k-peft1-test.jsonl'
# peft2 = '/userhomes/philhoon/kt-ai-challenge/result/kor-gsm8k-peft2-test.jsonl'

# ICL
# zs = '/userhomes/philhoon/kt-ai-challenge/result/zero-shot-public_mwp_data_v2_preprocess'
# zs_cot = '/userhomes/philhoon/kt-ai-challenge/result/zero-shot-cot-public_mwp_data_v2_preprocess'
# one_shot_cot = '/userhomes/philhoon/kt-ai-challenge/result/one-shot-cot-result-ko-gsm8k-test.jsonl'


In [7]:
class model_api():
    def __init__(self, url, headers):
        self.url = url
        self.headers = headers
        
    def generate(self, body):
        
        try:
            response = requests.post(self.url, data=body, headers=self.headers, verify=False)
        
        except:
            print("retrying due to an error......")
            time.sleep(20)
            return self.generate(body)
        
        return response
        
        

In [8]:
model = model_api(api['url'], api['headers'])
BODY_TEMP = api['body']

In [9]:
def get_gsm8k_zs3(que):
    prompt = f'다음 질문에 차근차근 답하시오. {que}'
    return prompt

In [10]:
def get_body(prompt, body_temp):
    body_temp["utterance"] = prompt
    return json.dumps(body_temp)

In [11]:
# def get_gsm8k_zs3(que):
#     prompt = f'다음 질문에 차근차근 답하시오. {que}'
#     return prompt

In [12]:
## Test dataset

# GSM8K 8792 (7473/1319)
gsm8k = '/userhomes/philhoon/kt-ai-challenge/data/openai-gpt-3.5-turbo-0301-2048-0.0-KOR-gsm8k-test.jsonl'

# MWP_KR_DATA 3750
mwp_kr = '/userhomes/philhoon/kt-ai-challenge/data/public_mwp_data_v2_reasoning-test.jsonl'

# AddSub 395
addsub = '/userhomes/philhoon/kt-ai-challenge/eng_data/kor-D_addsub-test.json'

# MultiArith 600
multiarith = '/userhomes/philhoon/kt-ai-challenge/eng_data/kor-D_multiarith-test.json'

# SingleEq 395
single_eq = '/userhomes/philhoon/kt-ai-challenge/eng_data/kor-D_single_eq-test.json'

In [13]:
gsm8k_data = utils.read_jsonlines(gsm8k)
mwp_kr_data = utils.read_jsonlines(mwp_kr)
addsub_data = utils.read_jsonlines(addsub)
multiarith_data = utils.read_jsonlines(multiarith)
single_eq_data = utils.read_jsonlines(single_eq)

In [14]:
print(len(addsub_data))
print(len(multiarith_data))
print(len(single_eq_data))

print(len(mwp_kr_data))
print(len(gsm8k_data))

79
120
102
720
1319


In [15]:
# random_indices

In [16]:
# pprint(single_eq_data[0])

In [17]:
print(gsm8k_data[0])
print(gsm8k_data[0])
print(gsm8k_data[0])
print(gsm8k_data[0])

{'question': "Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?", 'answer': 'Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.\nShe makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.\n#### 18', 'kor_question': '제넷의 오리는 하루에 16개의 알을 낳습니다. 그녀는 매일 아침 3개를 먹고, 매일 4개를 사용하여 친구들을 위해 머핀을 굽습니다. 그녀는 남은 알을 매일 농부 시장에서 1개당 $2에 판매합니다. 그녀는 매일 농부 시장에서 얼마나 벌까요? \n\nAnswer: 그녀는 매일 농부 시장에서 $16에서 $12를 뺀 $4를 벌게 됩니다.', 'kor_answer': '제넷은 하루에 16 - 3 - 4 = 9개의 오리 알을 판매합니다.\n그녀는 농부 시장에서 매일 9 * 2 = $18을 벌어들입니다.', 'answer_s': '18'}
{'question': "Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in doll

In [18]:
print(len(gsm8k_data))
print(len(mwp_kr_data))
print(len(addsub_data))
print(len(multiarith_data))
print(len(single_eq_data))

1319
720
79
120
102


In [19]:
data = multiarith_data

In [20]:
# data[0]

In [21]:
method = 'zs'

In [22]:
cnt = 0
for ins in tqdm(data):
    if 'kor_question' in ins:
        kor_que = ins['kor_question']
    else:
        kor_que = ins['question']
    
    
    if method == 'zs':
        prompt = kor_que
    elif method == 'zs-cot':
        prompt = get_gsm8k_zs3(kor_que)
    body = get_body(prompt, BODY_TEMP)
    
#     kor_que = ins['question']
#     body = get_body(kor_que, BODY_TEMP)
    
    response = model.generate(body)
    res_context = response.json()['data'][0]['result'][0]

    
#     ins['zs_infer'] = res_context
    print(f"prompt : \n{prompt}")
    print(f"response : \n{res_context}")
    print(f"----------------------------")
#     print(f"reason : \n{ins['reason']}")
    print(f"answer : \n{ins['answer']}")
    print(f"=============================")
    
    
    if cnt == 10:
        break
    else:
        cnt += 1
#     jsonl_writer.write_json_line(ins)

  0%|                                                                                                                                                | 0/120 [00:00<?, ?it/s]/userhomes/philhoon/miniconda3/envs/kt/lib/python3.8/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aiapi.genielabs.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
  1%|█▏                                                                                                                                      | 1/120 [00:01<03:18,  1.67s/it]

prompt : 
루크는 게임 컬렉션을 확장하려고 했습니다. 그는 친구로부터 게임을 2개 샀고, 중고장터에서 또 2개를 샀습니다. 만약 그 중 2개의 게임이 작동하지 않았다면, 그는 최종적으로 몇 개의 좋은 게임을 가지게 되었을까요?
response : 
그는 친구로부터 게임을 2개 샀습니다. 그 중 2개가 작동하지 않았다면
----------------------------
answer : 
2


/userhomes/philhoon/miniconda3/envs/kt/lib/python3.8/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aiapi.genielabs.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
  2%|██▎                                                                                                                                     | 2/120 [00:05<05:53,  3.00s/it]

prompt : 
레이첼은 색칠 공부를 위해 책을 두 권 샀습니다. 한 권에는 23개의 그림이 있고, 다른 한 권에는 32개의 그림이 있었습니다. 일주일이 지난 후에 그녀는 이미 44개의 그림을 색칠했습니다. 그녀가 아직 색칠해야 할 그림은 몇 개인가요?
response : 
"그녀는 일주일 후에 44개의 그림을 색칠했으므로
----------------------------
answer : 
11


/userhomes/philhoon/miniconda3/envs/kt/lib/python3.8/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aiapi.genielabs.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
  2%|███▍                                                                                                                                    | 3/120 [00:08<05:54,  3.03s/it]

...

prompt : 
학교의 도서 박람회에서 샘은 모험 소설 13권과 미스터리 소설 17권을 샀습니다. 그 중 15권은 중고였습니다. 그렇다면 그는 새로운 책을 몇 권 샀을까요?
response : 
"먼저
----------------------------
answer : 
15


/userhomes/philhoon/miniconda3/envs/kt/lib/python3.8/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aiapi.genielabs.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
  8%|███████████▎                                                                                                                           | 10/120 [00:30<04:25,  2.41s/it]

prompt : 
학교 급식소는 학생들의 점심을 위해 빨간 사과 33개와 초록 사과 23개를 주문했습니다. 그러나 과일을 원하는 학생은 21명뿐이었습니다. 급식소는 최종적으로 얼마나 많은 사과를 남겼을까요?
response : 
"빨간 사과는 33개
----------------------------
answer : 
35


/userhomes/philhoon/miniconda3/envs/kt/lib/python3.8/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aiapi.genielabs.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
  8%|███████████▎                                                                                                                           | 10/120 [00:34<06:24,  3.50s/it]

prompt : 
휴가 중에 레이첼은 동물원에서 6장의 사진을 찍었고 박물관에서 9장을 찍었습니다. 그 후에 그 중 11장을 삭제했다면, 그녀가 여행에서 가지고 있는 사진은 몇 장인가요?
response : 
"레이첼은 휴가 중에 동물원에서 6장을 찍었고 박물관에서 9장을 찍었습니다. 그 후에 그 중 11장을 삭제했으므로
----------------------------
answer : 
4


In [ ]:
Reasoning
다음 질문에 차근차근 답하시오. 힐트 여사는 책상에서 물 분수까지의 거리를 측정했습니다. 그 거리는 30 피트였습니다. 만약 그녀가 오늘 물 분수에 4번 가게 된다면, 힐트 여사는 총 몇 피트를 걸을까요?
response : 
그녀는 물 분수에 4번 갈 것입니다. 그녀는 물 분수에 4*30=90 피트를 갈 것입니다.



In [ ]:
zs_cot_data = utils.read_jsonlines(zs_cot)
zs_data = utils.read_jsonlines(zs)

In [ ]:
print(len(zs_cot_data))
len(zs_cot_data) == len(zs_data)

In [ ]:
from pprint import pprint

In [ ]:
cnt = 0
for ins in zs_data:
#     pprint(ins)
    print(f"question :\n{ins['question']}")
#     print(f"zs_cot_infer :\n{ins['zs_cot_infer']}")
    print(f"zs_infer :\n{ins['zs_infer']}")
    print(f"answer :\n{ins['answer']}")
    print("===========================")
    cnt += 1
    if cnt == 50:
        break